# Prague Marathon Results
## Data Analysis
#### David Koubek, Jiri Zelenka

In this data analysis, we load the 24 datasets that the scraper scraped from the RunCzech website. Then we treat/clean the data from missing values, preprocess it (adding Gender column inferred from "Age category", visualise various statistics about the data, and finally analyse the finishing times, trying to do just a basic rough prediction when the 2 hour marathon mark could be broken in the future. We have just 24 years, so it is not really enough observations to extrapolate anything into the future. Nonetheless, regarding the nature of sports, it is rather safe to guess that the physical boundaries of human body will be further pushed through precise training methods and improving nutrition and regeneration. The 2 hour mark is just a minute and a half away (although our Prague marathon doesn't host the world times and is a little behind). At the time of writing, the best time of 2h:01m:39s was achieved in Berlin marathon in 2018.

### Content of Analysis:

#### Import of Required Packages
##### Setup for the basic graphs
#### Data Loading + Merging
-------------------------------------
#### Preprocessing:

###### 1 Renaming Columns
###### 2 Cleaning Time + Adding Hours Column
###### 3 Cleaning Names of Runners + Completing Missing Ranks
###### 4 Adding Gender + Additional Genders 
-------------------------------------
#### Unique Countries, Names and Ages
#### Runners who Repeated Marathon
#### Top 5 Countries with the Most of Runners
#### Percentage of Czechs Over Time
-------------------------------------
#### Pivot Tables - Year x Gender
###### Min rank, Min time, Median time
###### Men x Women boxplots
-------------------------------------
#### Histograms 1995,2019, Total 
-------------------------------------
#### Predictions, 2 hour goal
-------------------------------------
-------------------------------------

### Import of Required Packages

In [ ]:
import pandas as pd                            # for dataframe
import numpy as np                             # working with arrays
import matplotlib.pyplot as plt                # Basic plots
import seaborn as sns                          # Plots, boxplots
import gender_guesser.detector as gender       # Gender predictions
from collections import Counter                # Counting items in set
from sklearn.linear_model import LinearRegression  # Liear model and predictions
from scipy import stats                        # Liear model and predictions

##### Setup  for the basic graphs



In [ ]:
plt.rcParams["figure.figsize"]=(14,8)

### Data loading + Merging

#### 1995 - 2019 (excluding 2012)

In [ ]:
years = [1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
         2005, 2006, 2007, 2008, 2009, 2010, 2011, 
         2013, 2014, 2015, 2016, 2017, 2018, 2019]

# 2012 is missing !!!, 24 runs


df_all = []
path = '../Scraper/Data_Marathons_Prague/data_'

for i in years:
    df = pd.read_csv(path + str(i) + '.csv')
    df["Year"] = i
    df_all.append(df)
    

df_merged = pd.concat(df_all, ignore_index=True)   
# Reseting Index to obtain overall indexes

len(df_merged)
# Total number of records = 94 299

In [ ]:
df_merged.head()
df_merged.tail()

# First glimpses into dataset

## Preprocessing

#### 1 Renaming Columns
#### 2 Cleaning Time + Adding Hours Column#### 2 
#### 3 Cleaning Names of Runners + Completing Missing Ranks
#### 4 Adding Gender + Additional Genders 


### 1 Renaming Columns


In [ ]:
df_renamed =df_merged.copy()

df_renamed = df_renamed.rename(columns={"Age cat.":"Age", "Chip time":"Time","St. number":"Start_number"  })

df_renamed.head()
df_renamed.tail()

### 2 Cleaning Time + Adding Hours Column
###### We deleted 288 observations with blank time, mostly from year 1998.
###### These were not suitable for further analysis

In [ ]:
# Deleting 288 observations with time = "-" - mostly from year 1998, reason of this = ???

df_times = df_renamed.copy()
df_times = df_times[df_renamed.Time != "-"]

df_times.reset_index(drop = True,inplace=True)

# Check for non-"-" times
times = df_times["Time"]
times.describe()

len(df_times)

# Keep working with 94 011 Runners

### 2.2 Adding Min Column

In [ ]:
# Counting minutes from Total time

tot_min = []
times = list(times)

for i in times:
    t=i.split(':')
    tot_min.append(int(t[0])*60+int(t[1])*1 +int(t[2])/60)
    
# len(tot_min)
tot_min = pd.DataFrame(tot_min)

df_mins = pd.concat([df_times, tot_min], axis=1)
df_mins = df_mins.rename(columns={0:"tot_mins"})

In [ ]:
# List of roudned minutes

Mins = list(np.around(np.array(tot_min),0))
Mins = pd.DataFrame(Mins)

df_mins = pd.concat([df_mins, Mins], axis=1)
df_mins = df_mins.rename(columns={0:"Mins"})

df_mins.head()
# df_mins.tail()

# Keep working with Dataframe including rounded minutes

###  3 Cleaning Names of Runners + Completing Missing Ranks

In [ ]:
all_names = list(df_mins.Name)

print(sorted(df_mins)[0:500])
print(sorted(df_mins)[len(df_mins)-500:len(all_names)])

    # In the head we can see some names starting with spaces or dots - we will transform them in next step
    # In the tail there are names of Russian, Israeli, Chiense or Japanese Runners
    # There are not so many of them, we keep them in original form

In [ ]:
df_named = df_mins.copy()

indd = df_named.index[df_named['Name'] == ' Peter Puškár                   '].tolist()
df_named.loc[indd,'Name'] = "Peter Puškár"
indd = df_named.index[df_named['Name'] == ' Radek Podracký               '].tolist()
df_named.loc[indd,'Name'] = "Radek Podracký"
indd = df_named.index[df_named['Name'] == ' STEINIVAR'].tolist()
df_named.loc[indd,'Name'] = "STEINIVAR"
indd = df_named.index[df_named['Name'] == ',Jiří Stránský Judr.'].tolist()
df_named.loc[indd,'Name'] = "Jiří Stránský Judr."
indd = df_named.index[df_named['Name'] == '.Jiří Nekuža'].tolist()
df_named.loc[indd,'Name'] = "Jiří Nekuža"
indd = df_named.index[df_named['Name'] == '- 50008873'].tolist()
df_named.loc[indd,'Name'] = "- -"
indd = df_named.index[df_named['Name'] == '- 50044654'].tolist()
df_named.loc[indd,'Name'] = "- -"
indd = df_named.index[df_named['Name'] == '?IVIND JOHANSEN'].tolist()
df_named.loc[indd,'Name'] = "IVIND JOHANSEN"
indd = df_named.index[df_named['Name'] == '\u202aesty rosenberg\u202c\u200f'].tolist()
df_named.loc[indd,'Name'] = "aesty rosenberg"
indd = df_named.index[df_named['Name'] == '\u202aron gashri\u202c\u200f'].tolist()
df_named.loc[indd,'Name'] = "aron gashri"
indd = df_named.index[df_named['Name'] == 'ＨＩＲＯＫＯ ＫＵＳＵＮＯＫＩ'].tolist()
df_named.loc[indd,'Name'] = "Hiroko Kusunoki"
indd = df_named.index[df_named['Name'] == 'ＨＩＲＯＫＯ ＳＨＩＮＯＭＩＹＡ'].tolist()
df_renamed.loc[indd,'Name'] = "Hiroko Shinomiya"
indd = df_named.index[df_named['Name'] == 'ＨＩＲＯＭＩ ＹＯＮＥＺＵ'].tolist()
df_named.loc[indd,'Name'] = "Hiromi Yonezu"
indd = df_named.index[df_named['Name'] == 'ＪＵＮＫＯ ＮＩＳＨＩＯＫＡ'].tolist()
df_named.loc[indd,'Name'] = "Junko Nishioka"
indd = df_named.index[df_named['Name'] == 'ＫＩＳＡＫＯ ＴＯＫＯ'].tolist()
df_named.loc[indd,'Name'] = "Kisako Toko"
indd = df_named.index[df_named['Name'] == 'ＫＩＹＯＫＯ ＫＡＷＡＳＡＫＩ'].tolist()
df_named.loc[indd,'Name'] = "Kiyoko Kawasaki"
indd = df_named.index[df_named['Name'] == 'ＭＡＳＡＫＯ ＯＮＩＳＨＩ'].tolist()
df_named.loc[indd,'Name'] = "Masako Onishi"
indd = df_named.index[df_named['Name'] == 'ＭＩＥＫＯ ＴＳＵＪＩＯＫＡ'].tolist()
df_named.loc[indd,'Name'] = "Mieko Tsujioka"

In [ ]:
df_named['Name'] = [str(i).lower() for i in df_named['Name']] 
# Changing all Names to lowercase
    
df_named.head()

In [ ]:
len(df_named)

In [ ]:
df_named[df_named["Rank"]=="-"]

### 3.2 Completing Missing Ranks

The only missing ranks are important for year 1997.
The rest of the missing ranks has missing time, too, and it will be dropped at all.

In [ ]:
df_ranks = df_named.copy()
df_ranks = df_ranks[df_ranks["Time"] != "-"]
# Dropping observaitons of 1998 having no times

missing_ranks = list(range(1601,1640))
missing_ranks = list(map(str, missing_ranks))
# Changing range of ranks into Strings

len(missing_ranks)
len(df_ranks.loc[df_ranks['Rank']=="-","Rank"])
# Check of the length of source and target => OK

df_ranks.loc[df_ranks['Rank']=="-", "Rank"] = missing_ranks
# Applying new ranks into the missing places

df_ranks["Rank"] = pd.to_numeric(df_ranks["Rank"])
# This improved our further analysis of Pivot tables, some Ranks were not in number format

df_ranks[df_ranks["Year"]==1997].tail(40)
# Check of the new Ranks

### 4 Adding Gender + Additional Genders

In [ ]:
df_gend = df_ranks.copy()

df_gend["Gender"] = df_gend["Age"].astype(str).str[0]
# Adding Gender column based on the first letter of Age code 

df_gend.tail()

### 4.2 Additional Genders
###### Now we have around 3.300 Runners with missing gender.
###### We will deal with them using gender.Detector() function

In [ ]:
Forename = []
for i in df_gend["Name"]:
    nick = i.split(' ')
    Forename.append(nick[0].capitalize())

df_gend["Forename"]=Forename

df_gend.reset_index(drop = True, inplace = True)

df_gend[df_gend["Gender"]=="-"].head()

# Here we generated column of Fornames with capitalized first letters

In [ ]:
d = gender.Detector()

# This package has satisfying results for both czech and foreign names,
# e.g. Jiří, Ehud, Jindřiška = OK,
# Saša, Dong = both possible genders

for i in range(len(df_gend)):
    #print(i)
    if df_gend.loc[i,"Gender"]=="-":

        new_gender = d.get_gender(df_gend.loc[i,"Forename"])
           
        if new_gender == "male":
            df_gend.loc[i, "Gender"] = "M"
                
        elif new_gender == "female":
            df_gend.loc[i, "Gender"] = "W"

# This part of code generated male/female/other labels,
# which we transformed into M and W labels

len(df_gend[df_gend["Gender"]=="-"])
#There are only 341 Genders missing

In [ ]:
df_gend[df_gend["Year"]==1995].head()

Additionally, we decided to label manually Turbo Tummo,
who won the run in 1995, but due to his unusual name he would stand out of our analysis.

In [ ]:
df_gend.loc[0, "Gender"] = "M"

df_final = df_gend.copy()

df_final.head()

#### Now we are done with preprocessing
From 3309 missing Genders we now miss only 340 observations of Gender, which is a great improvement!

### Unique Countries, Names and Ages

In [ ]:
#All unique countries:

uq_countries = df_final.Nationality.unique()   
len(uq_countries)

#uq_countries = 205

In [ ]:
# All unique ages:

uq_ages = df_final.Age.unique()
len(uq_ages)

# uq_ages = 15

In [ ]:
# All unique names:

uq_names = df_final.Name.unique()
len(uq_names)

# uq_names = 66087

### Runners who Repeated Marathon

Here we computed number of repetitions of each name over years
Anyway, we cannot yet say, how many of them are the same name in one run like "Petr Svoboda" 41x during 24 runs,
but there are definitely Runners who took more runs like "jack joseph michel brossaud" - 7x

In [ ]:
all_names = list(df_final.Name)
num_of_runs = Counter(all_names)

#num_of_runs.most_common() 
    #This is a list of all Names and their amount of appearence

### Top 5 Countries of origins (icluding Kenya)

In [ ]:
all_names = list(df_final.Nationality)
num_of_countries = Counter(all_names)
# This part assigns number of Runners to all countries

numerous_countries = num_of_countries.most_common(50)
numerous_countries = [i[0] for i in numerous_countries]
idx = numerous_countries.index("KEN")
# In this part we obtained index of Kenya in top 50 most 'populated' countries

numerous_countries = num_of_countries.most_common(4)
numerous_countries = [i[0] for i in numerous_countries]
numerous_countries.append("KEN")
# Now we took 4 countries with the most Runners, Kenya was added for comparison as the on with he best Runners

[num_of_countries.most_common(4), (num_of_countries.most_common()[idx])]

In [ ]:
years = [1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
         2005, 2006, 2007, 2008, 2009, 2010, 2011, 
         2013, 2014, 2015, 2016, 2017, 2018, 2019]

years = years[4:]
# Now we start from 1999 because of missing Nationalities in 1995-1998

countries_best =[]

for j in numerous_countries:
    
    nation_tot = []

    for i in years:
        nation = df_final[(df_final["Year"]== i) & (df_final["Nationality"]== j)]
        nation_min = nation.Mins.min()
        nation_tot.append(nation_min)
        
    countries_best.append(nation_tot)   
    
# This code looped over 5 countries and years 1999-2019 and obtained the best times for each country

In [ ]:
for i in range(len(countries_best)):
    plt.plot(years, countries_best[i])

plt.legend(numerous_countries, loc='upper right')

plt.show()

#### Kenya leads the running times, Czechia holds quiet good results, probably because of the huge amount of Runners. Italy is the only one of the most participating countries, who beat Czechia (in 2015).  

### Percentage of Czechs Over Time

##### After inspecting each year, we noticed of the occurence of "TCH" instead of "CZE" in Nationality column.

###### 1995 - no nationalities
###### 1996 2x CZE + 0x TCH
###### 1997 1x CZE + 727x TCH
###### 1998 1x CZE + 1045x TCH
###### 1999 986x CZE + 0x TCH

In [ ]:
# Replacing all "TCH" with "CZE"

df_final['Nationality'] = df_final['Nationality'].replace('TCH', 'CZE') 

In [ ]:
# Percents of czech Runners over years 1997-2019

cz_percent = []
for year in years:
    # We keep working with 1997-2019 years set
    
    cz = len(df_final[(df_final["Nationality"] == "CZE") & (df_final["Year"] == year)])
    alls = len(df_final[df_final["Year"] == year])
    per = cz/alls
    cz_percent.append(per)   
    
# cz_percent

In [ ]:
czechs = df_final[df_final["Nationality"] == "CZE"]
len(czechs)/len(df_final)

# Overall percentage of czech Runners = 46.2 %

plt.plot(years, cz_percent)
plt.show()

### Pivot tables Year x Gender

##### Min Rank, Min Time, Median Time

In [ ]:
# This table shows the top ranks of M and W of the ovreall ranks in year
# Here we can see that the top women took place about 20 of all the Runners and are moving forward to better positions

df_genders = df_final[df_final["Gender"] != "-"]
pd.pivot_table(df_genders,index=["Gender"],columns = ["Year"], values="Rank",aggfunc=np.min)

In [ ]:
# This table presents average times of W + M over years
# Non-clear Genders were excluded

df_genders = df_final[df_final["Gender"] != "-"]
pd.pivot_table(df_genders,index=["Gender"],columns = ["Year"], values="Mins",aggfunc=np.mean)

In [ ]:
# This table presents minimum times of W + M over years
# Non-clear Genders was excluded

df_genders = df_final[df_final["Gender"] != "-"]
pd.pivot_table(df_genders,index=["Gender"],columns = ["Year"], values="Mins",aggfunc=np.min)

In [ ]:
# In this part we calculate best and median times for both Men and Women

df_man = df_final[df_final["Gender"] == "M"]
df_woman = df_final[df_final["Gender"] == "W"]

years = years[1:]

# 2012 is missing, 1995 is excluded because of no Genders, 23 runs

best_w = []

for i in years:
    new_w = df_woman[df_woman["Year"]== i]
    min_new_w = new_w.Mins.min()
    best_w.append(min_new_w)

best_m = []

for i in years:
    new_m = df_man[df_man["Year"]== i]
    min_new_m = new_m.Mins.min()
    best_m.append(min_new_m)
   

# We do not calculate min of All, as the man times are the min of all times
#-----------------------------------------
    
median_w = []

for i in years:
    new_w = df_woman[df_woman["Year"]== i]
    median_new_w = new_w.Mins.median()
    median_w.append(median_new_w)

median_m = []

for i in years:
    new_m = df_man[df_man["Year"]== i]
    median_new_m = new_m.Mins.median()
    median_m.append(median_new_m)
    
    
median_all = []

for i in years:
    new_all2 = df_final[df_final["Year"]== i]
    median_new_all = new_all2.Mins.median()
    median_all.append(median_new_all)   

       

In [ ]:
# Now we plot Median times

plt.plot(years, median_w)
plt.plot(years, median_m)
plt.plot(years, median_all)

plt.legend(['Median Woman','Median Man', "Median All"], loc='upper left')
plt.show()

In [ ]:
# Top times evolution

plt.plot(years, best_m)
plt.plot(years, best_w)

plt.legend(['best Man','best Woman'], loc='upper right')
plt.show()

### Man x Woman boxplots

In [ ]:
sns.boxplot(x='Year', y='Mins', data=df_man)

# This plot includes all the times over years

In [ ]:
sns.boxplot(x='Year', y='Mins', data=df_woman)

In [ ]:
# In this plot there are both Women and Men over years 

plt.rcParams["figure.figsize"]=(18,12)

sns.boxplot(x="Year", y="Mins",
            hue="Gender", palette=["b", "r"],
            data=df_genders)
sns.despine(offset=10, trim=True)

### Histograms
###### Years 1995, 2019 and Total

In [ ]:
hist_1995 = df_final[df_final["Year"] == 1995].Mins
plt.hist(hist_1995, bins=30)

This Histogram doesn't produce perfect normal distribution due to less amount of Runners and the time limitation of 5 hours.

In [ ]:
hist_2019 = df_final[df_final["Year"] == 2019].Mins
plt.hist(hist_2019, bins=30)

In [ ]:
hist_all = df_final.Mins
plt.hist(hist_all, bins=30)

In this case, there is possible bias, because there were different limitations for maximal time of Run over years (5-7 hours)

### Predictions
##### In our Analysis we tried to figure out, whether and when Runners will break 2 hour goal (120 minutes of Run)

In [ ]:
years = [1995, 1996,1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
         2005, 2006, 2007, 2008, 2009, 2010, 2011, 
         2013, 2014, 2015, 2016, 2017, 2018, 2019]

# 2012 missing

data_pred = []

for i in years:
    new_pred = df_final[df_final["Year"]== i]
    min_data_pred = new_pred.Mins.min()
    data_pred.append(min_data_pred)
data_pred

In [ ]:
X = years
y = data_pred

gradient,intercept,r_value,p_value,std_err=stats.linregress(X,y)

In [ ]:
print("R-squared",r_value**2)

In [ ]:
gradient

In [ ]:
data = pd.DataFrame({'year': X,
                     'value': y})

p = sns.regplot(x='year',y='value',data=data, fit_reg=True)

# Here we get linear regression with standard errors

### Two Hour Goal
##### In this plot we rtry to predict whether the Runners will achieve their time better than 2 hours.
##### If they followed this simple linear trend, it would happen around 2054.

In [ ]:
plt.rcParams["figure.figsize"]=(20,6)

plt.plot(years[0:len(years)-5], best_m)
plt.plot([1996, 2065], [intercept + 1996*gradient, intercept+2065*gradient])

plt.axhline(y=120, color='r', linestyle='-')
# plt.legend(['best Man'], loc='upper right')

plt.show()